### Подготовка текста для анализа
Делаем лемматизацию с помощью библиотеки natasha, удаляем стоп-слова

In [1]:
from natasha import MorphVocab, Doc, NewsNERTagger, NewsMorphTagger, NewsEmbedding, Segmenter
import re, os

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)
morph_tagger = NewsMorphTagger(emb)

In [2]:
with open('real_news.txt') as f:
    text = f.read()

In [3]:
#Делаем предобработку (это может занять время)

doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [6]:
#Делаем лемматизацию
lemmas = []

for token in doc.tokens:
    token.lemmatize(morph_vocab)
    lemmas.append(token.lemma)

### Убираем стоп-слова

In [7]:
with open("NLTK's list of english stopwords.txt") as f:
    stop_words = f.read()
    
stop_words = stop_words.split('\n')    
    
clean_lemmas = []

for lemma in lemmas:
    if lemma not in stop_words and re.fullmatch("[\w\-]+", lemma) and lemma != '-':
        clean_lemmas.append(lemma)

### Строим граф совместной встречаемости слов
<p>Собираем пары слов и считаем для каждой пары частоту.</p>
<p>Откуда берутся пары и что такое совместная встречаемость?
*Это предложение для примера*. Получаем пары: Это предложение - предложение для - для примера</p>

In [8]:
from collections import Counter

In [9]:
pairs = []

for i in range(len(clean_lemmas)-1):
    #min и max нужны, чтобы слова в паре всегда стояли в алфавитном порядке
    #это позволит считать "это предложение" и "предложение это" за одну пару
    #т.е. совместную встречаемость слов "это" и "предложение"
    pair = min(clean_lemmas[i], clean_lemmas[i+1]) + ',' + max(clean_lemmas[i], clean_lemmas[i+1])
    pairs.append(pair) 

counts = Counter(pairs).most_common()

### Фильтруем пары с низкой частотой

In [10]:
fitered_counts = []

n = 10 #частота, по которой отсекаем низкочастотные пары, можно указать другое значение

for count in counts:
    if count[-1] > n:
        fitered_counts.append(count)

### Записываем результат в csv-файл

In [11]:
filename = "natasha_keywords_graph_filtered.csv"

csv = 'source,target,weight' + '\n'

for count in fitered_counts:
    line = count[0] + ',' + str(count[1]) + '\n'
    csv += line

with open(filename, 'w', encoding='utf-8') as f:
    f.write(csv)